In [11]:
from bs4 import BeautifulSoup
import requests
session = requests.session()
import json

In [12]:
RETRY_COUNT = 1

In [13]:
class IncomeTaxReturn:
    
    def __init__(self,acknum,mobile_num):
        
        self.mobile_num = mobile_num
        self.acknum = acknum
    
    def save_entity_page(self,mobile_num,acknum):
        url_link = "https://eportal.incometax.gov.in/iec/servicesapi/saveEntity"
        
        headers = {

            "User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36",
            "Referer":"https://eportal.incometax.gov.in/iec/foservices/",
            "Origin":"https://eportal.incometax.gov.in",
            "Host":"eportal.incometax.gov.in",
            "Content-Type": "application/json",
            "sn": "itrStatusService",
            "Accept":"application/json, text/plain, */*",
            "Accept-Encoding":"gzip, deflate, br",
            "Accept-Language":"en-GB,en-US;q=0.9,en;q=0.8",
            "Cache-Control":"no-cache",
            "Connection":"keep-alive",
            "Content-Length":"96",
        }

        payload ={
            "serviceName":"itrStatusService",
            "ackNum":acknum,
            "mobNum":mobile_num,
            "isdCd":"91"
        }
        res = session.post(url_link,headers = headers,json= payload)
        if res.status_code == 200:
            return res
        else:
            return "No Response"
        

    def parse_with_lxml(self,home_page_response):
        soup = BeautifulSoup(home_page_response.text,'lxml')
        return soup
    
    def get_smsTx_from_parsed_data(self,soup):
        
        start_index = soup.text.find("smsTx")+len("smsTx")+3
        smsTx = soup.p.text[start_index:-2]
        return smsTx
    
    def validate_otp(self,acknum,mobile_num,smsTx,otp):
        
        url_link = "https://eportal.incometax.gov.in/iec/servicesapi/validateOTP"
    
        headers = {

            "User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36",
            "Referer":"https://eportal.incometax.gov.in/iec/foservices/",
            "Origin":"https://eportal.incometax.gov.in",
            "Host":"eportal.incometax.gov.in",
            "Content-Type":"application/json"
        }

        payload = {

                    "serviceName":"itrStatusService",
                    "ackNum":acknum,
                    "mobNum":mobile_num,
                    "isdCd":"91",
                    "otp":otp,
                    "smsTx":smsTx
        }

        res = session.post(url_link,headers = headers,json= payload)
        if res.status_code == 200:
#             print(res.content)
            return res
        else:
#             print(res)
            return "No Response"
        
    def extract_data(self,soup):
        #print(soup.content)
        response = {}
        
        if soup.content == b'621':
            response = {"message":"Return Processed and Refund Paid"}
        else:
            response = {"Error":"The OTP entered is incorrect, Enter correct OTP."}
        
        return json.dumps(response,indent=4)
        
    def get_output(self):
        
        for retry_num in range(RETRY_COUNT):
            self.session = requests.session()
            
            sec_response = self.save_entity_page(self.mobile_num,self.acknum)
            #print(sec_response)
            
            #we parse the fetched html data using the lxml
            parsed_data = self.parse_with_lxml(sec_response)
            #print(parsed_data)
            
            smsTx = self.get_smsTx_from_parsed_data(parsed_data)
            #print(smsTx)
            
            otp = input("Enter the OTP Recieved: ")
            
            third_Resp = self.validate_otp(self.acknum,self.mobile_num,smsTx,otp)
            #print(third_Resp)
            
            output = self.extract_data(third_Resp)    
            return output

In [14]:
if __name__ == '__main__':
    print(IncomeTaxReturn("335842150290716","9916015591").get_output())

Enter the OTP Recieved: 032246
{
    "message": "Return Processed and Refund Paid"
}
